<div class="row">
  <div class="column">
    <img src="./img/logo-onera.png" width="200">
  </div>
  <div class="column">
    <img src="./img/logo-ISAE_SUPAERO.png" width="200">
  </div>
</div>

# Custom Modules Management

This tutorial aims to guide you in the management of your customs modules and in the resolution of a simple problem using FAST-OAD. 
The problem under consideration consists in a cantilever beam of length **L** and whose plain rectangular section is described by its height **h** and width **l**. A force **F** is applied at the tip of the beam that also undergoes its proper weight homogeneously distributed **w**. 

The objectives this tutorial will be to create and call custom modules through FAST-OAD to solve the following problems:
* For **F** = 1000N, **L** = 2.0m, **l** = 0.3m find the height **h** that leads to a displacement of 0.02m considering an aluminium beam. 
* For **F** = 1e5N, **L** = 10.0m, **l** = 0.3m find the height **h** so that the maximum normal stress within the beam do not exceed the aluminium yield stress (450e6 Pa).

<div class="row">
  <div class="column">
    <img src="./img/problem_description.png" width="600">
  </div>
</div>

## Imports

In [ ]:
import os.path as pth
import logging
import fastoad.api as oad

## Directories and files definition 

In [ ]:
DATA_FOLDER = "data"

WORK_FOLDER = "workdir"

CONFIGURATION_FILE_NAME = pth.join(DATA_FOLDER, "beam_problem.yml")

logging.basicConfig(level=logging.INFO, format='%(levelname)-8s: %(message)s')

## 1. Custom Modules

The following modules are created: [section_properties.py](./modules/section_properties.py), [weight.py](./modules/weight.py), [displacements.py](./modules/displacements.py), [stresses.py](./modules/stresses.py). They respectively compute the heigth of the beam for a given moment of inertia and width, the linear weight **w** from material density and section dimensions, the necessary moment of inertia to reach a given displacements and the necessary moment of inertia not to overcome yield stress. 

As you may notice, each module is registered using the decorator `@oad.RegisterOpenMDAOSystem`. 

An example is provided for a geometric module that compute the height of the beam for a given moment of inertia and a given width: 

<div class="row">
    <div class="column">
        <img src="./img/custom_module_ex.gif" width="600">
  </div>
</div>

## 2. Configuration file
Once the modules are created, the problem can be defined through the [configuration file](./data/beam_problem.yml): 
* First, the folder containing your custom module must be indicated within the `module_folders` section.
* Then, your problem is defined within the `model` section. 
* If variables are shared between different modules (models) then loops occurs and `nonlinear_solver` and `linear_solver` must be defined, here non linear Block Gauss-Seidel and linear Direct solvers are used. 
The configuration file for the first problem is shown below. As you can notice, three different modules are called sequentially: the geometry, the weight and the displacements. As Displacements and geometry shares inputs/outputs we need to introduce solvers. 

<div class="row">
  <div class="column">
    <img src="./img/disp_config.gif" width="800">
  </div>
</div>

Once your configuration file is generated, you can check your models connections using the n2 diagram: 

In [ ]:
oad.write_n2(CONFIGURATION_FILE_NAME, overwrite=True)

<div class="row">
  <div class="column">
    <img src="./img/partition_tree_n2.svg" width="800">
  </div>
</div>

## 3. Input file 

One the configuration is generated you can generate or use an already generated input file: 

In [ ]:
# UNCOMMENT THE FOLLOWING LINE TO GENERATE A BLANK INPUT FILE 
#oad.generate_inputs(CONFIGURATION_FILE_NAME, overwrite=True)
INPUT_FILE_NAME = pth.join(DATA_FOLDER, "problem_inputs.xml")
oad.variable_viewer(INPUT_FILE_NAME)

## 4. Run the problem 

Once your configuration is defined and inputs checked, you can run the problem 

In [ ]:
disp_problem = oad.evaluate_problem(CONFIGURATION_FILE_NAME, overwrite=True)

In [ ]:
oad.variable_viewer(disp_problem.output_file_path)

## 5. Change models 

If you want to solve an other problem, for example the second one, you may have to replace one or several modules. Here again everything happens in the [configuration file](./data/beam_problem_stress.yml).

You have to replace the `displacements` model by the `stresses` one as illustrated below (be sure that module id has been updated accordingly). You may save the configuration under another name to solve the two problem consecutively.

<div class="row">
  <div class="column">
    <img src="./img/stress_config.gif" width="800">
  </div>
</div>

Then generate the input file and update the input date if necessary. 

In [ ]:
CONFIGURATION_FILE_STRESS_NAME = pth.join(DATA_FOLDER, "beam_problem_stress.yml")
# UNCOMMENT THE FOLLOWING LINE TO GENERATE A BLANK INPUT FILE 
#oad.generate_inputs(CONFIGURATION_FILE_STRESS_NAME, overwrite=True)
INPUT_FILE_NAME = pth.join(DATA_FOLDER, "problem_inputs_stress.xml")
oad.variable_viewer(INPUT_FILE_NAME)

In [ ]:
stress_problem = oad.evaluate_problem(CONFIGURATION_FILE_STRESS_NAME, overwrite=True)

In [ ]:
oad.variable_viewer(stress_problem.output_file_path)